In [72]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LinearRegression

In [73]:
df = pd.read_csv('/content/life_expectancy_data.csv')

In [74]:
df = df.iloc[:, 3:]

In [75]:
#penultimo numeor de la matricula=3, 2, 3 - Todas las variables, menos X2, X6, X10, X14, X18
columns_to_drop = [2, 6, 10, 14, 17]

# Drop columns
df = df.drop(df.columns[columns_to_drop], axis=1)

In [76]:
df.isnull().sum()

Life expectancy            10
Adult Mortality            10
Alcohol                   194
percentage expenditure      0
Hepatitis B               553
 BMI                       34
under-five deaths           0
Polio                      19
Diphtheria                 19
 HIV/AIDS                   0
GDP                       448
 thinness  1-19 years      34
 thinness 5-9 years        34
Schooling                 163
dtype: int64

In [77]:
#hay muchas columnas con datos faltantes y son necesarios para hacer algunos ejercios,
#por ejemplo el LinearRegression, para solucionar esto vamos a imputar los valores faltantes

In [78]:
X = df.drop(columns=['Life expectancy '])
y = df['Life expectancy ']

In [79]:
from sklearn.impute import SimpleImputer

# Crear un imputador que reemplace los NaN con la media de la columna
imputer = SimpleImputer(strategy='mean')

# Aplicar la imputación a tu conjunto de características X
X_imputed = imputer.fit_transform(X)
y_imputed = imputer.fit_transform(y.values.reshape(-1, 1))
y_imputed = y_imputed.flatten()

In [80]:
modelo = LinearRegression()

In [81]:
from sklearn.model_selection import cross_val_score
puntuaciones = cross_val_score(modelo, X_imputed, y_imputed, cv=5, scoring='r2')  # R-cuadrado como métrica de evaluación

In [82]:
print("Puntuaciones de validación cruzada:", puntuaciones)

Puntuaciones de validación cruzada: [0.81022965 0.80098836 0.69379987 0.74728199 0.78726378]


In [83]:
promedio_puntuaciones = puntuaciones.mean()
desviacion_estandar_puntuaciones = puntuaciones.std()


In [84]:
print(f"Puntuación promedio: {promedio_puntuaciones:.2f}")
print(f"Desviación estándar de las puntuaciones: {desviacion_estandar_puntuaciones:.2f}")

Puntuación promedio: 0.77
Desviación estándar de las puntuaciones: 0.04


In [85]:
X.shape

(2938, 13)

### filter

In [86]:
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score

# Lista para almacenar las métricas de evaluación
metricas = []

# Rango de números de predictores a probar
num_predictores_range = range(1, X.shape[1] + 1)

#  imputador que reemplace los NaN con la media de la columna
imputer = SimpleImputer(strategy='mean')
X_imputed = imputer.fit_transform(X)
y_imputed = imputer.fit_transform(y.values.reshape(-1, 1))
y_imputed = y_imputed.flatten()

for num_predictores in num_predictores_range:
    # Utilizar SelectKBest para seleccionar las mejores características
    selector = SelectKBest(score_func=f_regression, k=num_predictores)
    X_selected = selector.fit_transform(X_imputed, y_imputed)

    modelo = LinearRegression()

    # Calculamos r^2 utilizando validación cruzada
    puntuaciones = cross_val_score(modelo, X_selected, y_imputed, cv=5, scoring='r2')


    metricas.append(puntuaciones.mean())

#  número óptimo de predictores
num_predictores_optimos = num_predictores_range[metricas.index(max(metricas))]


print(f"Número óptimo de predictores: {num_predictores_optimos}")

# métricas de evaluación por número de predictores
print("Métricas de evaluación por número de predictores:")
for i, metrica in enumerate(metricas):
    print(f"Número de predictores: {i + 1}, R^2: {metrica:.2f}")

# nombres de las columnas seleccionadas
columnas_seleccionadas = X.columns[selector.get_support()]
print("Nombres de las columnas seleccionadas:")
print(columnas_seleccionadas)

X_optimo = X.iloc[:, :num_predictores_optimos]


Número óptimo de predictores: 9
Métricas de evaluación por número de predictores:
Número de predictores: 1, R^2: 0.51
Número de predictores: 2, R^2: 0.69
Número de predictores: 3, R^2: 0.71
Número de predictores: 4, R^2: 0.75
Número de predictores: 5, R^2: 0.77
Número de predictores: 6, R^2: 0.78
Número de predictores: 7, R^2: 0.78
Número de predictores: 8, R^2: 0.78
Número de predictores: 9, R^2: 0.78
Número de predictores: 10, R^2: 0.78
Número de predictores: 11, R^2: 0.78
Número de predictores: 12, R^2: 0.77
Número de predictores: 13, R^2: 0.77
Nombres de las columnas seleccionadas:
Index(['Adult Mortality', 'Alcohol', 'percentage expenditure', 'Hepatitis B',
       ' BMI ', 'under-five deaths ', 'Polio', 'Diphtheria ', ' HIV/AIDS',
       'GDP', ' thinness  1-19 years', ' thinness 5-9 years', 'Schooling'],
      dtype='object')


### Wrapper

In [87]:
from sklearn.feature_selection import SequentialFeatureSelector

In [88]:
imputer = SimpleImputer(strategy='mean')
X_imputed = imputer.fit_transform(X)
y_imputed = imputer.fit_transform(y.values.reshape(-1, 1))
y_imputed = y_imputed.flatten()

modelo = LinearRegression()

sfs = SequentialFeatureSelector(modelo, n_features_to_select=None, cv=5)
sfs.fit(X_imputed, y_imputed)

predictores_optimos = X.columns[sfs.support_]

# nombres de las columnas seleccionadas
print("Predictores óptimos encontrados por el método:")
print(predictores_optimos)

/usr/local/lib/python3.10/dist-packages/sklearn/feature_selection/_sequential.py:211: FutureWarning: Leaving `n_features_to_select` to None is deprecated in 1.0 and will become 'auto' in 1.3. To keep the same behaviour as with None (i.e. select half of the features) and avoid this warning, you should manually set `n_features_to_select='auto'` and set tol=None when creating an instance.
  warnings.warn(


Predictores óptimos encontrados por el método:
Index(['Adult Mortality', ' BMI ', 'Diphtheria ', ' HIV/AIDS', 'GDP',
       'Schooling'],
      dtype='object')


### Filter-Wrapper

In [89]:
from sklearn.feature_selection import RFE

In [90]:
# Inicializa el método
rfe = RFE(estimator=modelo, n_features_to_select=None)

# selecciónamos las características recursivas
rfe.fit(X_imputed, y_imputed)

# predictores óptimos
predictores_optimos = X.columns[rfe.support_]

# nombres de las columnas seleccionadas
print("Predictores óptimos encontrados por el método:")
print(predictores_optimos)

Predictores óptimos encontrados por el método:
Index(['Alcohol', ' BMI ', 'Diphtheria ', ' HIV/AIDS', ' thinness  1-19 years',
       'Schooling'],
      dtype='object')


### K-vecinos más cercanos

In [91]:
from sklearn.model_selection import cross_val_score
from sklearn.impute import SimpleImputer
from sklearn.neighbors import KNeighborsRegressor
from sklearn.feature_selection import SelectKBest, f_regression
import numpy as np

In [92]:
metricas = []

num_predictores_range = range(1, X.shape[1] + 1)

imputer = SimpleImputer(strategy='mean')
X_imputed = imputer.fit_transform(X)
y_imputed = imputer.fit_transform(y.values.reshape(-1, 1))
y_imputed = y_imputed.flatten()

for num_predictores in num_predictores_range:
    #  SelectKBest para seleccionar las mejores características
    selector = SelectKBest(score_func=f_regression, k=num_predictores)
    X_selected = selector.fit_transform(X_imputed, y_imputed)

    modelo = KNeighborsRegressor()  # modelo KNN

    puntuaciones = cross_val_score(modelo, X_selected, y_imputed, cv=5, scoring='r2')

    metricas.append(puntuaciones.mean())

num_predictores_optimos = num_predictores_range[metricas.index(max(metricas))]

X_optimo = X_imputed[:, :num_predictores_optimos]

print(f"Número óptimo de predictores: {num_predictores_optimos}")
print("Métricas de evaluación por número de predictores:")
for i, metrica in enumerate(metricas):
    print(f"Número de predictores: {i + 1}, R^2: {metrica:.2f}")

columnas_seleccionadas = X.columns[:num_predictores_optimos]
print("Nombres de las columnas seleccionadas:")
print(columnas_seleccionadas)


Número óptimo de predictores: 4
Métricas de evaluación por número de predictores:
Número de predictores: 1, R^2: 0.32
Número de predictores: 2, R^2: 0.88
Número de predictores: 3, R^2: 0.82
Número de predictores: 4, R^2: 0.89
Número de predictores: 5, R^2: 0.88
Número de predictores: 6, R^2: 0.88
Número de predictores: 7, R^2: 0.88
Número de predictores: 8, R^2: 0.88
Número de predictores: 9, R^2: 0.78
Número de predictores: 10, R^2: 0.78
Número de predictores: 11, R^2: 0.76
Número de predictores: 12, R^2: 0.73
Número de predictores: 13, R^2: 0.73
Nombres de las columnas seleccionadas:
Index(['Adult Mortality', 'Alcohol', 'percentage expenditure', 'Hepatitis B'], dtype='object')


In [93]:
import statsmodels.api as sm
# Aqui solo vemos el summary de nuestro modelo de pura curiosidad para ver que tanto se ajusta a los datos
modelo_final = KNeighborsRegressor()
modelo_final.fit(X_optimo, y_imputed)

X_optimo_con_intercept = sm.add_constant(X_optimo)  # Agregar una constante (intercepto)
modelo_stats = sm.OLS(y_imputed, X_optimo_con_intercept).fit()
print(modelo_stats.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.586
Model:                            OLS   Adj. R-squared:                  0.585
Method:                 Least Squares   F-statistic:                     1037.
Date:                Wed, 06 Sep 2023   Prob (F-statistic):               0.00
Time:                        05:39:36   Log-Likelihood:                -9490.3
No. Observations:                2938   AIC:                         1.899e+04
Df Residuals:                    2933   BIC:                         1.902e+04
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         70.3785      0.493    142.816      0.0

In [94]:
df = pd.read_csv('/content/life_expectancy_data.csv')
df = df.iloc[:, 2:]
#penultimo numeor de la matricula=3, 2, 3 - Todas las variables, menos X2, X6, X10, X14, X18
columns_to_drop = [2, 6, 10, 14, 17]

# Drop columns at specific locations (indices)
df = df.drop(df.columns[columns_to_drop], axis=1)
df.head()

,Status,Life expectancy,infant deaths,Alcohol,percentage expenditure,Measles,BMI,under-five deaths,Total expenditure,Diphtheria,HIV/AIDS,Population,thinness 1-19 years,Income composition of resources,Schooling
0,Developing,65.0,62,0.01,71.279624,1154,19.1,83,8.16,65.0,0.1,33736494.0,17.2,0.479,10.1
1,Developing,59.9,64,0.01,73.523582,492,18.6,86,8.18,62.0,0.1,327582.0,17.5,0.476,10.0
2,Developing,59.9,66,0.01,73.219243,430,18.1,89,8.13,64.0,0.1,31731688.0,17.7,0.470,9.9
3,Developing,59.5,69,0.01,78.184215,2787,17.6,93,8.52,67.0,0.1,3696958.0,17.9,0.463,9.8
4,Developing,59.2,71,0.01,7.097109,3013,17.2,97,7.87,68.0,0.1,2978599.0,18.2,0.454,9.5


In [95]:
df['Status'].unique()

array(['Developing', 'Developed'], dtype=object)

In [96]:
# Realizar la codificación one-hot para la columna 'status'
df = pd.get_dummies(df, columns=['Status'], drop_first=True)

# Separar las características (X) y la variable objetivo (y)
X = df.drop(columns=['Life expectancy '])  # Excluir la columna objetivo
y = df['Life expectancy ']

In [97]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import cross_val_score
from sklearn.impute import SimpleImputer

# Inicializar el modelo de regresión de árbol de decisión
modelo = DecisionTreeRegressor()

# Imputar valores faltantes en las características y la variable objetivo
imputer = SimpleImputer(strategy='mean')
X_imputed = imputer.fit_transform(X)
y_imputed = imputer.fit_transform(y.values.reshape(-1, 1))
y_imputed = y_imputed.flatten()

# Evaluar el modelo con validación cruzada utilizando R^2
puntuaciones_r2 = cross_val_score(modelo, X_imputed, y_imputed, cv=5, scoring='r2')

# Calcular la puntuación promedio de R^2
puntuacion_r2_promedio = puntuaciones_r2.mean()

# Evaluar el modelo con validación cruzada utilizando MSE
mse_scores = cross_val_score(modelo, X_imputed, y_imputed, cv=5, scoring='neg_mean_squared_error')

# Calcular el MSE promedio (cambiamos el signo para obtener el MSE positivo)
mse_promedio = -mse_scores.mean()

# Evaluar el modelo con validación cruzada utilizando MAE
mae_scores = cross_val_score(modelo, X_imputed, y_imputed, cv=5, scoring='neg_mean_absolute_error')

# Calcular el MAE promedio (cambiamos el signo para obtener el MAE positivo)
mae_promedio = -mae_scores.mean()

# Imprimir los resultados
print("Puntuación R^2 promedio de validación cruzada:", puntuacion_r2_promedio)
print("MSE promedio de validación cruzada:", mse_promedio)
print("MAE promedio de validación cruzada:", mae_promedio)

Puntuación R^2 promedio de validación cruzada: 0.7145081059430748
MSE promedio de validación cruzada: 25.83538244828179
MAE promedio de validación cruzada: 3.7123551849082324


Claro, puedo ayudarte a resumir las conclusiones a partir de los resultados obtenidos en este ejercicio:

1. **Consideras que el modelo de regresión lineal es adecuado para los datos. ¿Por qué?** Basado en los resultados obtenidos, parece que el modelo de regresión lineal no es la mejor opción para estos datos. El R^2 promedio de validación cruzada de aproximadamente 0.721 sugiere que, aunque el modelo puede explicar una parte significativa de la variabilidad en la esperanza de vida, aún deja una cantidad considerable de varianza sin explicar.

2. **¿Qué método de selección de características consideras que funciona bien con los datos? ¿Por qué?** El método de selección de características que funcionó mejor en este caso fue el "Filter-Wrapper"y (Wrapper). El metodo de filter nos ayudo a visualizar que teniendo de 6 a 11 variables vamos a tener el R^2 más alto(0.78). Entonces estos metodos nos seleciionaron las 6 caracteristicas mas relevantes.  si queremos un modelo con mas variables entonces ya podemos utilizar el metodo de filter porque ahi nos da 9 variables optimas. En realidad los 3 metodos funcionaron y al final utilizando los 3 podemos llegar a una mejor conclusión de cuantas y cuales caracteriticas usar.

3. **Características Sobresalientes:** Las características seleccionadas por el método "Filter-Wrapper" incluyen 'Alcohol', 'BMI', 'Diphtheria', 'HIV/AIDS', 'thinness 1-19 years' y 'Schooling'. Estas características pueden considerarse sobresalientes, ya que tienen un fuerte impacto en la predicción de la esperanza de vida según el modelo. Graficando las variables predictoras contra la variable de respuesta tambíen nos ayudan a visualizar cuales variables tienen un fuerte impacto. Las dos que más tienen relación con life exepectancy de acuerdo a las qqplot graphs es Alchcol y schooling.

4. **El modelo de regresión no lineal funcionó mejor que el lineal? ¿Por qué?** El modelo de regresión no lineal,  K-Vecinos Más Cercanos (KNN), alcanzó un R^2 de 0.89 con 4 predictores, superando el R^2 del modelo lineal. Esto sugiere que, en este caso, un enfoque no lineal podría ser más adecuado para modelar la relación entre las características y la esperanza de vida. El hecho de que el modelo no lineal sirvio mejor sugiere que 1) los datos tienen relaciones no lineales significativas entre las características y la variable de respuesta. 2)los datos se agrupan en regiones o grupos locales con tendencias similares. 3)Los datos contienen patrones y tendencias intrínsecamente no lineales que son difíciles de modelar con un enfoque lineal.

5. **Árbol de Decisión con Variable Categórica "Status":** La adición de la variable categórica "Status" al modelo de árbol de decisión no parece haber mejorado significativamente el rendimiento. Esto podría indicar que esta variable no aporta una mejora sustancial en la capacidad predictiva del modelo en comparación con las características numéricas ya consideradas.

6. **Conclusión General:** En general, este ejercicio revela que el modelo de regresión no lineal,  es el más efectivo para predecir la esperanza de vida en función de las características seleccionadas. Además, la combinación de métodos de selección de características, como Filter y Wrapper, puede ayudar a identificar las características más influyentes. Sin embargo, también es importante notar que el modelo de regresión lineal no es completamente inservible, y también puede ajustarse a estos datos con los que estuvimos trabajando

aqui hacemos una rapida visualizacion de las variables con la variable de respuesta. Nota importante: todavia no limpiamos los datos y como se puede apreciar varios de los datos estan vacios